In [3]:
# Import necessary libraries
import numpy as np                           # Numerical operations
import pandas as pd                          # Data loading and manipulation
import json                                  # Parsing GeoJSON strings
import folium                                # Interactive mapping
import joblib                                # Model loading
import geopandas as gpd                      # Geospatial data handling
from shapely.geometry import Point, box      # Geometry creation
import matplotlib.pyplot as plt              # Plotting

from sktime.classification.kernel_based import TimeSeriesSVC  # Time series SVM classifier
from sktime.dists_kernels import AggrDist       # Aggregated distance kernel wrapper
from sklearn.gaussian_process.kernels import RBF  # Radial basis function kernel
from sklearn.metrics import f1_score           # Classification metric


In [ ]:
#--- Data import and date parsing ---
# Load time series data from CSV
loaded_data = pd.read_csv('path/to/your/data.csv', delimiter=',')
# Extract date substring from 'system:index' and convert to datetime
loaded_data = loaded_data.assign(date=(loaded_data['system:index'].str[0:9]))
loaded_data['date'] = pd.to_datetime(loaded_data['date'])


In [8]:
#--- Generate unique ID from GeoJSON coordinates ---
def extract_coordinates(geojson_str):
    """
    Parse a GeoJSON POINT string and return a unique ID formatted as 'lat_lon'.
    """
    geo = json.loads(geojson_str)
    lon, lat = geo['coordinates']
    return f"{lat:.6f}_{lon:.6f}"

# Apply coordinate extraction to create ID column
loaded_data['ID'] = loaded_data['.geo'].apply(extract_coordinates)

In [9]:
#--- Filter out cloudy/snowy observations ---
filtered_data = loaded_data[loaded_data['NDSI'] < 0.4]

In [ ]:
#--- Load trained classifier and feature configuration ---
model_data = joblib.load('path/to/your/model.pkl')
clf = model_data['model']
selected_features = model_data['features']
multivariate_combinations = model_data['multivariate_combinations']

print('Loaded features:', selected_features)
print('Available feature combinations:', list(multivariate_combinations.keys()))

In [ ]:
#--- Choose specific feature combination for prediction ---
combination_name = 'NDVI_EVI_FAPAR'
if combination_name not in multivariate_combinations:
    raise ValueError(f"Combination {combination_name} not found.")
features_to_use = multivariate_combinations[combination_name]
print('Using features:', features_to_use)

In [ ]:
#--- Prepare data for prediction ---
# Subset filtered_data to only selected features plus ID and date
data_for_pred = filtered_data[features_to_use + ['ID','date']]
# Set a MultiIndex (ID, date) for time series input
X_new = data_for_pred.set_index(['ID','date']).sort_index()
print('Shape of prediction data:', X_new.shape)

In [ ]:
#--- Predict cluster/label for each time series ID ---
y_pred = clf.predict(X_new)
unique_ids = X_new.index.get_level_values('ID').unique()
print('Number of unique IDs:', len(unique_ids))

In [14]:
# Create DataFrame of predictions by ID
predictions_df = pd.DataFrame({'ID': unique_ids, 'predicted': y_pred})
# Merge predictions back into filtered_data
filtered_with_pred = filtered_data.merge(predictions_df, on='ID', how='left')

In [ ]:
#--- Extract latitude and longitude for mapping ---
def extract_lat_lon(geojson_str):
    """
    Parse a GeoJSON POINT string and return (lat, lon).
    """
    geo = json.loads(geojson_str)
    lon, lat = geo['coordinates']
    return lat, lon

# Apply extraction for mapping
filtered_with_pred['lat'], filtered_with_pred['lon'] = zip(*filtered_with_pred['.geo'].apply(extract_lat_lon))
print(filtered_with_pred[['lat','lon','predicted']].head())

In [16]:
#--- Aggregate predictions to one per ID ---
agg = filtered_with_pred.groupby('ID').agg(
    predicted=('predicted', lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]),
    lat=('lat','first'),
    lon=('lon','first')
).reset_index()

In [ ]:
#--- Create GeoDataFrame of point centres ---
gdf = gpd.GeoDataFrame(
    agg,
    geometry=gpd.points_from_xy(agg.lon, agg.lat),
    crs='EPSG:4326'
)

In [ ]:
#--- Create square polygons around each point ---
pixel_size = 10  # size of square in meters
# Reproject to metric CRS for accurate square size
gdf_metric = gdf.to_crs(epsg=32633)
# Generate square polygons of specified pixel size
gdf_metric['geometry'] = gdf_metric.geometry.apply(lambda p: box(p.x - pixel_size/2, p.y - pixel_size/2,
                                                               p.x + pixel_size/2, p.y + pixel_size/2))
# Save square-pixel shapefile
pix_shp = 'model_pixels.shp'
gdf_metric.to_file(pix_shp, driver='ESRI Shapefile')
print(f"Square pixel shapefile saved to {pix_shp}")